# Using the Google Drive in Python and openEO environment

In WEED we want to use files from the GoogleDrive for the processing - mainly in the benchmarking part of the project. MOreover, some of the results of the benchmarking should be directly uploaded to the GDrive and therefore available for all user. The following steps had to be implemented to access the GDrive from Python.

1. Sign in to the WEED Google Account and open the Google Cloud Console to create a project. <br>
  - go to https://console.cloud.google.com/  <br>
  - Create a New Project: Click on “Select a project” at the top and then “New Project”. Give it a name and create. Name is "WEED-2024"
2. enable the Google Drive API for the WEED project <br>
  - Navigate to APIs & Services: In the left sidebar, go to “APIs & Services” > “Dashboard”.
  - Enable APIs: Click on “Enable APIs and Services”. Search for “Google Drive API” and enable it for your project.
3. create credentials
  - Generate Credentials: In the left sidebar, go to “APIs & Services” > “Credentials”.
  - Create Credentials: Click “Create Credentials” > “Service Account”. Fill in details, choose a role, and create a JSON key. Save this JSON key securely.
    - Name: gdrive_access
    - Email: gdrive-access@weed-2024.iam.gserviceaccount.com (automatic generation)
    - enable the service account status
4. JSON access key
  - Download and Use JSON Key: Store the downloaded JSON key securely on your local machine. This key will be used for authenticating your application to access the Drive API. (The service account key can only be retrieved the first time)
  - BEST: store the key in a secure location like the VITO valut and access it from there client-side (https://confluence.vito.be/pages/viewpage.action?spaceKey=EP&title=Vault+user+guide)
  - Therefore, if you need a new key just add a new key to the gdrive_access credentials!!!!!
5. make sure that for each file and folder which should be available in Python the servie account email address is added to the "access" of these files and folders. Currently only the folder "openeo_tests" under WEED/working/WP4_ToolboxDvlpt is granted access in this way.
6. install the pydrive2 python package in the WEED environment

In [1]:
import platform
import os

from pydrive2.auth import GoogleAuth
from pydrive2.fs import GDriveFileSystem
import pandas as pd
import hvac
from getpass import getpass

In [ ]:
def get_gdrive_credentials(username: str = 'buchhornm') -> str:
    """
    Retrieves Google Drive access credentials from Terrascope VAULT using LDAP authentication.

    This method prompts the user to enter their password for Terrascope VAULT, authenticates
    with the VAULT using LDAP, and fetches the Google Drive service account credentials
    from a specified KV storage path.

    :param username: LDAP username used to authenticate with the VAULT, defaults to 'buchhornm'
    :return: Google Drive access credentials
    """
    password_prompt = 'Please enter your password for the Terrascope VAULT: '
    service_account_password = getpass(prompt=password_prompt)

    client = hvac.Client(url='https://vault.vgt.vito.be')

    client.auth.ldap.login(
        username=username,
        password=service_account_password,
        mount_point='ldap'
    )

    secret_version_response = client.secrets.kv.v2.read_secret_version(mount_point='kv',
                                                                       path='apps/weed/google-service-account')

    return secret_version_response['data']['data']['gdrive-access']

In [ ]:
# get the credentials for the GDrive service account access from the vault
gdrive_credentials = get_gdrive_credentials(username='buchhornm')

In [2]:
# set needed credential path depending on operation system
if platform.system() == "Windows":
    path_cred = os.path.normpath(r'\\netapp03.vgt.vito.be\MEPVM\Marcel\weed\weed-2024-23dfc6e2b884.json')
elif platform.system() == "Linux":
    path_cred = os.path.normpath(r'/data/MEPVM/Marcel/weed/weed-2024-23dfc6e2b884.json')
else:
    raise SystemError("Running on an unsupported platform")

In [3]:
# init the fsspec filesystem to access the files & folders available for the service account credentials
# gdrive-access@weed-2024.iam.gserviceaccount.com
# "1k27bitdRp41AtHq1xupyqwKaTLzrMUMu" is the ID of the only folder currently available for this service account
# if more folder should be available then add the email_address to the user of the files and/or folders wished
# see: https://filesystem-spec.readthedocs.io/en/latest/usage.html#use-a-file-system for more info to interact with file system

gdrive = GDriveFileSystem("1k27bitdRp41AtHq1xupyqwKaTLzrMUMu",
                          use_service_account=True,
                          client_json_file_path=path_cred)

In [4]:
# list files and folders
for root, dnames, fnames in gdrive.walk(gdrive.root):
    print(root, dnames, fnames)

1k27bitdRp41AtHq1xupyqwKaTLzrMUMu [] ['basic_20x20km_grid_no-crs.gpkg', 'SK_v5_reference-points_EUNIS2012.csv', 'global_high-res_UTMzones.gpkg', 'Land_masses_10km_buffered_Clipped_S2.gpkg']


In [5]:
# we can interact now quite easy with the data - example for the CSV file read in pandas
# Note: you always have to start from the entrance point and then add the sub-folder plus filenames separated by "/"
with gdrive.open(gdrive.root + "/" + 'SK_v5_reference-points_EUNIS2012.csv', 'rb') as f:
    df = pd.read_csv(f)
df.head()


,fid,entry,x,y,valid_L1,valid_L2,valid_L3,lon,lat,label_L1,label_L2,label_L3
0,1,P5000001,4.849642e+06,2.811280e+06,True,True,True,17.118680,48.185489,C,C3,C3.4
1,2,P5000002,4.851750e+06,2.815627e+06,True,True,True,17.152425,48.222568,C,C3,C3.4
2,3,P5000003,4.854840e+06,2.816960e+06,True,True,True,17.195542,48.231791,C,C3,C3.4
3,4,P5000004,4.854535e+06,2.819285e+06,True,True,True,17.194425,48.252869,C,C3,C3.4
4,5,P5000005,4.855180e+06,2.819730e+06,True,True,True,17.203641,48.256284,C,C3,C3.4
